# * Ex-D Data

## Parameter

In [1]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb
import re
import FN_Monitoring as fn

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

pd.set_option('future.no_silent_downcasting', True)

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')

## Actual : Monitoring
->  STG_KPI_NEWCO_DIAMOND_ACTUAL_INC

In [2]:
''' Execute Summany Data '''


# Input parameter
curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
# v_start_date = 20250201
print(f'\nParameter input...\n')
# print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
print(f'\n{TDMDBPR_db} : Connected')
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        CASE WHEN REGEXP_LIKE(METRIC_CD, '^DB1R000900|^DB1R001000|^DB1S000101|^DB1S000109|^DB2R000500|^DB2S000100') 
            OR METRIC_CD IN (
                'DB0R00010001' --Total Inflow M1 : DTAC'
                , 'DB1S000100' --Prepay New sub
                , 'DB2S010100' --Postpaid activation sub B2C
                , 'DB2S020100' --Postpaid activation sub B2B
                --, 'DB1S000102' --Prepaid Activation Subs : Thai Mass : DTAC
                --, 'DB1S000103' --Prepaid Activation Subs : AEC/Migrants : DTAC
                --, 'DB1S000104' --Prepaid Activation Subs : Tourists (IN + OUT) : DTAC
                ) THEN 'Obsolete'
            ELSE 'Active'
            END KPI_FLAG
        , PAR_KEY, BASENAME, METRIC_CD, METRIC_NAME
        , CASE WHEN REGEXP_LIKE(METRIC_CD, 'A[A-K]$') THEN SUBSTR(METRIC_CD, -2) ELSE 'ALL' END CHANNEL_CD
        , MAX(LOAD_DATE) LOAD_DATE, MIN(TM_KEY_DAY) START_DAY, MAX(TM_KEY_DAY) END_DAY
        , SUM(CASE WHEN AREA_CD = 'P' THEN METRIC_VALUE END) AS P
        , SUM(CASE WHEN AREA_TYPE = 'G' THEN METRIC_VALUE END) AS G
        , SUM(CASE WHEN AREA_TYPE = 'H' THEN METRIC_VALUE END) AS H
        , SUM(CASE WHEN AREA_TYPE = 'HH' THEN METRIC_VALUE END) AS HH
        , SUM(CASE WHEN AREA_TYPE = 'CCAA' THEN METRIC_VALUE END) AS CCAA
        , SUM(CASE WHEN AREA_CD = '080' THEN METRIC_VALUE END) AS H_080
        , SUM(CASE WHEN AREA_CD = '016' THEN METRIC_VALUE END) AS H_016
        , SUM(CASE WHEN AREA_CD = '040' THEN METRIC_VALUE END) AS H_040
        , COUNT(DISTINCT AREA_TYPE) CNT_AREA_TYPE, COUNT(DISTINCT AREA_CD) CNT_AREA_CD, COUNT(1) ROW_CNT
    FROM GEOSPCAPPO.STG_KPI_NEWCO_DIAMOND_ACTUAL_INC
    GROUP BY PAR_KEY, BASENAME, METRIC_CD, METRIC_NAME
    --ORDER BY 1, PAR_KEY, BASENAME, METRIC_CD 
""")


try:
    execute_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
    print(f'\n   -> Execute query... {execute_datetime}')

    # Get : Summany DataFrame
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])
    print(f'\n   -> Summany DataFrame : {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()
    print(f'\n{TDMDBPR_db} : Disconnected')


Parameter input...


TDMDBPR : Connected

   -> Execute query... 2025-03-18, 11:29:27

   -> Summany DataFrame : 131 rows, 20 columns

TDMDBPR : Disconnected


In [3]:
''' Revenue '''

# curr_mth = chk_src_df['TM_KEY_MTH'].max()
src_d_kpi_flag = 'Active'
rev_metric_list = [
    'DB1R000100' #Prepay Revenue
    , 'DB2R000100' #Postpaid Revenue
    , 'DB2R010100' #Postpaid Revenue B2C
    , 'DB2R020100' #Postpaid Revenue B2B
    ]

src_d_rev_overall_df = chk_src_df.loc[chk_src_df['KPI_FLAG']==src_d_kpi_flag].copy()
# src_d_rev_overall_df = src_d_rev_overall_df.loc[src_d_rev_overall_df['TM_KEY_MTH']==curr_mth]
src_d_rev_overall_df = src_d_rev_overall_df.loc[src_d_rev_overall_df['METRIC_CD'].isin(rev_metric_list)]
src_d_rev_overall_df = src_d_rev_overall_df.groupby(['PAR_KEY', 'METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum', 'CCAA':'sum', 'H_080':'sum', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
src_d_rev_overall_df = src_d_rev_overall_df.fillna(0).sort_values(by=['METRIC_CD']).reset_index()

mod_col_list = src_d_rev_overall_df.iloc[:, 6:12].columns.tolist()
for col in mod_col_list:
    src_d_rev_overall_df[col] = src_d_rev_overall_df[col].apply(lambda x: format(x, ',.0f'))

src_d_rev_overall_df

,PAR_KEY,METRIC_CD,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,P,G,H,HH,CCAA,H_080,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,20250314,DB1R000100,Prepay Revenue,2025-03-14,20240528,20250314,"5,442,177,066","9,765,141,841","9,758,075,375","9,758,075,375","5,442,176,775","89,481,595",5,1100,103088
1,20250314,DB2R000100,Postpaid Revenue,2025-03-14,20240528,20250314,"8,294,347,681","14,784,590,081","14,772,332,631","14,772,332,631","8,294,347,266","229,793,741",5,1100,102531
2,20250314,DB2R010100,Postpaid Revenue B2C,2025-03-14,20240528,20250314,"7,537,820,565","13,450,698,072","13,440,227,178","13,440,227,178","7,537,820,150","213,894,100",5,1100,102475
3,20250314,DB2R020100,Postpaid Revenue B2B,2025-03-14,20240528,20250314,"756,527,117","1,333,892,009","1,332,105,452","1,332,105,452","756,527,117","15,899,642",5,1100,66009


In [4]:
''' Subscription '''

src_d_kpi_flag = 'Active'
sub_metric_list = [
    'DB1S000500' #Prepay Usage Subs
    , 'DB1S000600' #Prepay Revenue Subs
    , 'DB2S000500' #Postpaid Active Subs
    , 'DB2S010500' #Postpaid Active Subs B2C
    , 'DB2S020400' #Postpaid Active Sub B2B
    ]

src_d_sub_overall_df = chk_src_df.loc[chk_src_df['KPI_FLAG']==src_d_kpi_flag].copy()
src_d_sub_overall_df = src_d_sub_overall_df.loc[src_d_sub_overall_df['METRIC_CD'].isin(sub_metric_list)]
src_d_sub_overall_df = src_d_sub_overall_df.groupby(['PAR_KEY', 'METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum', 'CCAA':'sum', 'H_080':'sum', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
src_d_sub_overall_df = src_d_sub_overall_df.fillna(0).sort_values(by=['METRIC_CD']).reset_index()

mod_col_list = src_d_sub_overall_df.iloc[:, 6:12].columns.tolist()
for col in mod_col_list:
    src_d_sub_overall_df[col] = src_d_sub_overall_df[col].apply(lambda x: format(x, ',.0f'))

src_d_sub_overall_df

,PAR_KEY,METRIC_CD,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,P,G,H,HH,CCAA,H_080,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,20250314,DB1S000500,Prepay Usage Subs,2025-03-14,20240528,20250314,"1,107,382,462","1,976,095,329","1,974,721,309","1,974,721,309","1,107,382,306","18,949,078",5,1100,103073
1,20250314,DB1S000600,Prepay Revenue Subs,2025-03-14,20240528,20250314,"584,566,754","1,047,165,940","1,046,358,829","1,046,358,829","584,566,697","10,098,135",5,1100,103073
2,20250314,DB2S000500,Postpaid Active Subs,2025-03-14,20240528,20250314,"541,894,943","968,794,891","967,976,765","967,976,765","541,894,800","14,308,879",5,1100,102905
3,20250314,DB2S010500,Postpaid Active Subs B2C,2025-03-14,20240528,20250314,"478,614,259","855,183,921","854,541,163","854,541,163","478,614,116","12,903,757",5,1100,102905
4,20250314,DB2S020400,Postpaid Active Sub B2B,2025-03-14,20240528,20250314,"63,280,684","113,610,970","113,435,602","113,435,602","63,280,684","1,405,122",5,1100,102329


## Actual : Transaction
->  STG_KPI_NEWCO_DIAMOND_ACTUAL_INC

In [5]:
''' Execute Daily by case '''


# Input parameter
curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
v_start_date = 20240101
print(f'\nParameter input...\n')
print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
print(f'\n{TDMDBPR_db} : Connected')
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        PAR_KEY, METRIC_CD, METRIC_NAME, TM_KEY_DAY
        , MAX(LOAD_DATE) LOAD_DATE
        , SUM(CASE WHEN AREA_CD = 'P' THEN METRIC_VALUE END) AS P
        , SUM(CASE WHEN AREA_TYPE = 'G' THEN METRIC_VALUE END) AS G
        , SUM(CASE WHEN AREA_TYPE = 'H' THEN METRIC_VALUE END) AS H
        , SUM(CASE WHEN AREA_TYPE = 'HH' THEN METRIC_VALUE END) AS HH
        , SUM(CASE WHEN AREA_TYPE = 'CCAA' THEN METRIC_VALUE END) AS CCAA
        , SUM(CASE WHEN AREA_CD = '080' THEN METRIC_VALUE END) AS H_080
        , SUM(CASE WHEN AREA_CD = '016' THEN METRIC_VALUE END) AS H_016
        , SUM(CASE WHEN AREA_CD = '040' THEN METRIC_VALUE END) AS H_040
        , COUNT(DISTINCT AREA_TYPE) CNT_AREA_TYPE, COUNT(DISTINCT AREA_CD) CNT_AREA_CD, COUNT(1) ROW_CNT
    FROM GEOSPCAPPO.STG_KPI_NEWCO_DIAMOND_ACTUAL_INC
    WHERE METRIC_CD IN (
        'DB1R000100' --Prepay Revenue
        , 'DB2R000100' --Postpaid Revenue
        , 'DB2R010100' --Postpaid Revenue B2C
        , 'DB2R020100' --Postpaid Revenue B2B
        , 'DB1S000500' --Prepay Usage Subs
        , 'DB1S000600' --Prepay Revenue Subs
        , 'DB2S000500' --Postpaid Active Subs
        , 'DB2S010500' --Postpaid Active Subs B2C
        , 'DB2S020400' --Postpaid Active Sub B2B
        )
    AND TM_KEY_DAY >= {v_start_date}
    GROUP BY PAR_KEY, METRIC_CD, METRIC_NAME, TM_KEY_DAY
""")


try:
    execute_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
    print(f'\n   -> Execute query... {execute_datetime}')

    # Get : Summary DataFrame
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])
    print(f'\n   -> Daily DataFrame : {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()
    print(f'\n{TDMDBPR_db} : Disconnected')


Parameter input...

   -> v_start_date: 20240101



TDMDBPR : Connected

   -> Execute query... 2025-03-18, 11:30:23

   -> Daily DataFrame : 828 rows, 16 columns

TDMDBPR : Disconnected


In [6]:
''' DB1R000100 : Prepay Revenue '''

v_start_day = 20240101
v_metric_cd = 'DB1R000100'
print(f'\nDB1R000100 : Prepay Revenue\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_start_day: {v_start_day}')

DB1R000100_day__df = chk_src_df.copy()
DB1R000100_day__df = DB1R000100_day__df.loc[DB1R000100_day__df['TM_KEY_DAY']>=v_start_day]
DB1R000100_day__df = DB1R000100_day__df.loc[DB1R000100_day__df['METRIC_CD']==v_metric_cd]
DB1R000100_day__df = DB1R000100_day__df.fillna(0).sort_values(by=['METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = DB1R000100_day__df.iloc[:, 5:13].columns.tolist()
for col in mod_col_list:
    DB1R000100_day__df[col] = DB1R000100_day__df[col].apply(lambda x: format(x, ',.0f'))

DB1R000100_day__df


DB1R000100 : Prepay Revenue

   -> v_metric_cd: DB1R000100
   -> v_start_day: 20240101


,PAR_KEY,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,P,G,H,HH,CCAA,H_080,H_016,H_040,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,20250314,DB1R000100,Prepay Revenue,20240528,2024-06-06,"57,081,475","57,081,475","57,081,459","57,081,459","57,081,459",0,"303,238","890,374",5,1099,1111
1,20250314,DB1R000100,Prepay Revenue,20240529,2024-06-07,"54,750,640","54,750,640","54,750,601","54,750,601","54,750,601",0,"320,514","861,052",5,1099,1111
2,20250314,DB1R000100,Prepay Revenue,20240530,2024-06-08,"59,115,506","59,115,506","59,115,490","59,115,490","59,115,490",0,"312,960","934,515",5,1099,1111
3,20250314,DB1R000100,Prepay Revenue,20240531,2024-06-09,"70,858,400","70,858,400","70,858,378","70,858,378","70,858,378",0,"382,069","1,193,650",5,1099,1111
4,20250314,DB1R000100,Prepay Revenue,20240601,2024-06-10,"70,350,955","70,350,955","70,350,942","70,350,942","70,350,942",0,"385,655","1,166,659",5,1099,1111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,20250314,DB1R000100,Prepay Revenue,20250310,2025-03-14,"57,966,061","115,932,122","115,836,760","115,836,760","57,966,061","1,149,640",0,0,5,1098,1123
88,20250314,DB1R000100,Prepay Revenue,20250311,2025-03-14,"53,756,901","107,513,802","107,426,010","107,426,010","53,756,901","1,087,688",0,0,5,1098,1123
89,20250314,DB1R000100,Prepay Revenue,20250312,2025-03-14,"52,720,965","105,441,930","105,357,478","105,357,478","52,720,965","1,091,635",0,0,5,1098,1123
90,20250314,DB1R000100,Prepay Revenue,20250313,2025-03-14,"51,848,693","103,697,385","103,616,442","103,616,442","51,848,693","1,049,698",0,0,5,1098,1123


In [7]:
''' DB2R000100 : Postpaid Revenue '''

v_start_day = 20240101
v_metric_cd = 'DB2R000100'
print(f'\nDB2R000100 : Postpaid Revenue\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_start_day: {v_start_day}')

DB2R000100_day__df = chk_src_df.copy()
DB2R000100_day__df = DB2R000100_day__df.loc[DB2R000100_day__df['TM_KEY_DAY']>=v_start_day]
DB2R000100_day__df = DB2R000100_day__df.loc[DB2R000100_day__df['METRIC_CD']==v_metric_cd]
DB2R000100_day__df = DB2R000100_day__df.fillna(0).sort_values(by=['METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = DB2R000100_day__df.iloc[:, 5:13].columns.tolist()
for col in mod_col_list:
    DB2R000100_day__df[col] = DB2R000100_day__df[col].apply(lambda x: format(x, ',.0f'))

DB2R000100_day__df


DB2R000100 : Postpaid Revenue

   -> v_metric_cd: DB2R000100
   -> v_start_day: 20240101


,PAR_KEY,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,P,G,H,HH,CCAA,H_080,H_016,H_040,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,20250314,DB2R000100,Postpaid Revenue,20240528,2024-06-06,"93,542,212","93,542,212","93,542,212","93,542,212","93,542,212",0,"1,056,671","2,289,788",5,1099,1108
1,20250314,DB2R000100,Postpaid Revenue,20240529,2024-06-07,"281,666,951","281,666,951","281,666,951","281,666,951","281,666,951",0,"3,264,840","6,703,783",5,1099,1108
2,20250314,DB2R000100,Postpaid Revenue,20240530,2024-06-08,"6,554,985","6,554,985","6,554,985","6,554,985","6,554,985",0,"72,122","128,142",5,1099,1102
3,20250314,DB2R000100,Postpaid Revenue,20240531,2024-06-09,"6,595,697","6,595,697","6,595,697","6,595,697","6,595,697",0,"87,719","144,124",5,1099,1103
4,20250314,DB2R000100,Postpaid Revenue,20240601,2024-06-10,"8,733,426","8,733,426","8,733,426","8,733,426","8,733,426",0,"90,050","188,734",5,1099,1103
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,20250314,DB2R000100,Postpaid Revenue,20250310,2025-03-14,"92,896,558","185,793,116","185,638,602","185,638,602","92,896,558","3,155,328",0,0,5,1097,1115
88,20250314,DB2R000100,Postpaid Revenue,20250311,2025-03-14,"85,264,105","170,528,211","170,377,785","170,377,785","85,264,105","2,949,729",0,0,5,1098,1117
89,20250314,DB2R000100,Postpaid Revenue,20250312,2025-03-14,"84,910,210","169,820,420","169,665,240","169,665,240","84,910,210","2,942,186",0,0,5,1098,1117
90,20250314,DB2R000100,Postpaid Revenue,20250313,2025-03-14,"83,815,507","167,631,014","167,487,756","167,487,756","83,815,507","3,060,452",0,0,5,1098,1119


In [8]:
''' DB2R010100 : Postpaid Revenue B2C '''

v_start_day = 20240101
v_metric_cd = 'DB2R010100'
print(f'\nDB2R010100 : Postpaid Revenue B2C\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_start_day: {v_start_day}')

DB2R010100_day__df = chk_src_df.copy()
DB2R010100_day__df = DB2R010100_day__df.loc[DB2R010100_day__df['TM_KEY_DAY']>=v_start_day]
DB2R010100_day__df = DB2R010100_day__df.loc[DB2R010100_day__df['METRIC_CD']==v_metric_cd]
DB2R010100_day__df = DB2R010100_day__df.fillna(0).sort_values(by=['METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = DB2R010100_day__df.iloc[:, 5:13].columns.tolist()
for col in mod_col_list:
    DB2R010100_day__df[col] = DB2R010100_day__df[col].apply(lambda x: format(x, ',.0f'))

DB2R010100_day__df


DB2R010100 : Postpaid Revenue B2C

   -> v_metric_cd: DB2R010100
   -> v_start_day: 20240101


,PAR_KEY,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,P,G,H,HH,CCAA,H_080,H_016,H_040,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,20250314,DB2R010100,Postpaid Revenue B2C,20240528,2024-06-06,"90,417,597","90,417,597","90,417,597","90,417,597","90,417,597",0,"1,054,690","2,284,168",5,1098,1107
1,20250314,DB2R010100,Postpaid Revenue B2C,20240529,2024-06-07,"280,520,688","280,520,688","280,520,688","280,520,688","280,520,688",0,"3,262,211","6,695,846",5,1099,1108
2,20250314,DB2R010100,Postpaid Revenue B2C,20240530,2024-06-08,"4,883,710","4,883,710","4,883,710","4,883,710","4,883,710",0,"70,826","122,243",5,1099,1101
3,20250314,DB2R010100,Postpaid Revenue B2C,20240531,2024-06-09,"5,308,594","5,308,594","5,308,594","5,308,594","5,308,594",0,"86,421","134,865",5,1099,1101
4,20250314,DB2R010100,Postpaid Revenue B2C,20240601,2024-06-10,"5,843,787","5,843,787","5,843,787","5,843,787","5,843,787",0,"84,771","173,885",5,1099,1102
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,20250314,DB2R010100,Postpaid Revenue B2C,20250310,2025-03-14,"87,349,442","174,698,884","174,544,188","174,544,188","87,349,442","3,150,245",0,0,5,1097,1114
88,20250314,DB2R010100,Postpaid Revenue B2C,20250311,2025-03-14,"82,873,454","165,746,909","165,596,804","165,596,804","82,873,454","2,942,117",0,0,5,1098,1116
89,20250314,DB2R010100,Postpaid Revenue B2C,20250312,2025-03-14,"81,703,911","163,407,821","163,252,986","163,252,986","81,703,911","2,935,168",0,0,5,1098,1117
90,20250314,DB2R010100,Postpaid Revenue B2C,20250313,2025-03-14,"82,540,808","165,081,617","164,938,715","164,938,715","82,540,808","3,044,926",0,0,5,1098,1119


In [9]:
''' DB2R020100 : Postpaid Revenue B2B '''

v_start_day = 20240101
v_metric_cd = 'DB2R020100'
print(f'\nDB2R020100 : Postpaid Revenue B2B\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_start_day: {v_start_day}')

DB2R020100_day__df = chk_src_df.copy()
DB2R020100_day__df = DB2R020100_day__df.loc[DB2R020100_day__df['TM_KEY_DAY']>=v_start_day]
DB2R020100_day__df = DB2R020100_day__df.loc[DB2R020100_day__df['METRIC_CD']==v_metric_cd]
DB2R020100_day__df = DB2R020100_day__df.fillna(0).sort_values(by=['METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = DB2R020100_day__df.iloc[:, 5:13].columns.tolist()
for col in mod_col_list:
    DB2R020100_day__df[col] = DB2R020100_day__df[col].apply(lambda x: format(x, ',.0f'))

DB2R020100_day__df


DB2R020100 : Postpaid Revenue B2B

   -> v_metric_cd: DB2R020100
   -> v_start_day: 20240101


,PAR_KEY,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,P,G,H,HH,CCAA,H_080,H_016,H_040,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,20250314,DB2R020100,Postpaid Revenue B2B,20240528,2024-06-06,"3,124,614","3,124,614","3,124,614","3,124,614","3,124,614",0,"1,981","5,620",5,726,729
1,20250314,DB2R020100,Postpaid Revenue B2B,20240529,2024-06-07,"1,146,263","1,146,263","1,146,263","1,146,263","1,146,263",0,"2,629","7,936",5,734,737
2,20250314,DB2R020100,Postpaid Revenue B2B,20240530,2024-06-08,"1,671,275","1,671,275","1,671,275","1,671,275","1,671,275",0,"1,296","5,899",5,740,743
3,20250314,DB2R020100,Postpaid Revenue B2B,20240531,2024-06-09,"1,287,104","1,287,104","1,287,104","1,287,104","1,287,104",0,"1,298","9,258",5,780,783
4,20250314,DB2R020100,Postpaid Revenue B2B,20240601,2024-06-10,"2,889,639","2,889,639","2,889,639","2,889,639","2,889,639",0,"5,279","14,848",5,731,734
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,20250314,DB2R020100,Postpaid Revenue B2B,20250310,2025-03-14,"5,547,116","11,094,232","11,094,414","11,094,414","5,547,116","5,082",0,0,5,635,650
88,20250314,DB2R020100,Postpaid Revenue B2B,20250311,2025-03-14,"2,390,651","4,781,302","4,780,980","4,780,980","2,390,651","7,612",0,0,5,666,681
89,20250314,DB2R020100,Postpaid Revenue B2B,20250312,2025-03-14,"3,206,299","6,412,599","6,412,253","6,412,253","3,206,299","7,017",0,0,5,700,715
90,20250314,DB2R020100,Postpaid Revenue B2B,20250313,2025-03-14,"1,274,699","2,549,397","2,549,041","2,549,041","1,274,699","15,527",0,0,5,737,752


In [10]:
# txn_metric_list = [
#     'DB1R000100' #Prepay Revenue
#     , 'DB2R000100' #Postpaid Revenue
#     , 'DB2R010100' #Postpaid Revenue B2C
#     , 'DB2R020100' #Postpaid Revenue B2B
#     , 'DB1S000500' #Prepay Usage Subs
#     , 'DB1S000600' #Prepay Revenue Subs
#     , 'DB2S000500' #Postpaid Active Subs
#     , 'DB2S010500' #Postpaid Active Subs B2C
#     , 'DB2S020400' #Postpaid Active Sub B2B
#     ]